# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=9dac69f2e46c771817bc1b3782ebdde981fc6aaa52a6edb12546dd2a6989cbc2
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=tEgPvRm1RSqA6Xrdc19sPytgm7Ftty&prompt=consent&token_usage=remote&access_type=offline&code_chal

# Tratamento

In [ ]:
project_id = "repositoriodedadosgpsp"

query = """
SELECT ano, 'publico' as flag_publico_privado,variavel,categoria, media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos_2023` union all SELECT ano, flag_publico_privado,variavel, categoria,media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_flag_privados_2023`
"""

df = pandas_gbq.read_gbq(query, project_id=project_id)

# Mostra as primeiras linhas
df.info()

df=df[df['variavel']!='tipo_vinculo']

# 1. Carregar os dados originais
df_ipca = pd.read_csv('it-ipca-formatado.csv')

# 2. Criar listas com anos e valores
anos = df_ipca.columns.str.extract('(\d{4})')[0].tolist()
valores = df_ipca.iloc[0].tolist()

# 3. Criar novo DataFrame
df_ipca_transformado = pd.DataFrame({
    'Ano': anos,
    'IPCA_acumulado': valores
})

# 4. Limpeza dos dados
def converter_valor(valor):
    if isinstance(valor, str):
        if valor == '...':
            return None
        return float(valor.replace(',', '.'))
    return valor

df_ipca_transformado['IPCA_acumulado'] = df_ipca_transformado['IPCA_acumulado'].apply(converter_valor)

# 5. Remover linhas com valores nulos (opcional)
df_ipca_transformado = df_ipca_transformado.dropna()

# 6. Converter ano para inteiro
df_ipca_transformado['Ano'] = df_ipca_transformado['Ano'].astype(int)

# 7. Visualizar o resultado
print(df_ipca_transformado)

# 8. Salvar em novo arquivo CSV
df_ipca_transformado.to_csv('ipca_simplificado.csv', index=False)

df_ipca_transformado = df_ipca_transformado.rename(columns={'Ano': 'ano'})
df_ipca_transformado['ano'] = df_ipca_transformado['ano'].astype(int)
df['ano'] = df['ano'].astype(int)

df_completo = pd.merge(
    df,
    df_ipca_transformado,
    on='ano',
    how='left'
)

df_completo['fator_correcao'] = (7100.5 / df_completo['IPCA_acumulado'])

df_completo['media_remuneracao_ajustada']= df_completo['media_remuneracao'] * (df_completo['fator_correcao'])

df_completo['flag_publico_privado']=np.where(df_completo['flag_publico_privado']=="privado","Privado",df_completo['flag_publico_privado'])
df_completo['flag_publico_privado']=np.where(df_completo['flag_publico_privado']=="publico","Público",df_completo['flag_publico_privado'])
df_completo.head(2)

df_completo = df_completo[['ano','flag_publico_privado', 'variavel', 'categoria', 'media_remuneracao',
       'fator_correcao', 'media_remuneracao_ajustada']]




In [ ]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ano                         24 non-null     int64  
 1   flag_publico_privado        24 non-null     object 
 2   variavel                    24 non-null     object 
 3   categoria                   24 non-null     object 
 4   media_remuneracao           24 non-null     float64
 5   fator_correcao              24 non-null     float64
 6   media_remuneracao_ajustada  24 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


# Upload

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
bigquery.SchemaField('flag_publico_privado','STRING',description='Se se refere ao setor publico ou privado'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]

 ## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos_privados_2023') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df_completo, table_ref, job_config=job_config)
job.result()